In [26]:
x = SR.symbol('x', domain='real')
y = SR.symbol('y', domain='real')
z = SR.symbol('z', domain='real')
theta = SR.symbol('theta', domain='real')
phi = SR.symbol('phi', domain='real')
r = SR.symbol('r', domain='positive')
x_vec = vector([x, y, z])
r_vec = vector([theta, phi, r])
s_vec = vector([arctan(y/x), arccos(z / sqrt(x^2 + y^2 + z^2)), sqrt(x^2 + y^2 + z^2)])
c_vec = vector([r * sin(phi) * cos(theta), r * sin(phi) * sin(theta), r * cos(phi)])
s_to_c = dict({r: s_vec[0], theta: s_vec[1], phi: s_vec[2]})
c_to_s = dict({x: c_vec[0], y: c_vec[1], z: c_vec[2]})
s_j = jacobian(s_vec, x_vec)
c_j = jacobian(c_vec, r_vec)
h = (c_j.transpose() * c_j).simplify_full()
kappa = sqrt(h.det())
gamma = [
    [
        [
            sum([
                1 / 2 * h_inv[alpha, k] * (h[alpha, m].diff(r_vec[n]) 
                                   + h[alpha, n].diff(r_vec[m]) 
                                   + h[m, n].diff(r_vec[alpha])) for alpha in range(3)
            ]) for k in range(3)
        ] for n in range(3)
    ] for m in range(3)
]

r_hat = s_vec[0].gradient(x_vec).subs(c_to_s).simplify_full()
theta_hat = vector([-sin(theta), cos(theta), 0])
phi_hat = vector([cos(phi)*cos(theta), cos(phi)*sin(theta), -sin(phi)])
x_hat = vector([r_hat[0], theta_hat[0], phi_hat[0]])
y_hat = vector([r_hat[1], theta_hat[1], phi_hat[1]])
z_hat = vector([r_hat[2], theta_hat[2], phi_hat[2]])

r_x = s_vec[0].diff(x).subs(c_to_s).simplify_full()
r_y = s_vec[0].diff(y).subs(c_to_s).simplify_full()
r_z = s_vec[0].diff(z).subs(c_to_s).simplify_full()

theta_x = s_vec[1].diff(x).simplify_full().subs(c_to_s).simplify_full()
theta_y = s_vec[1].diff(y).subs(c_to_s).simplify_full()
theta_z = s_vec[1].diff(z)

phi_x = cos(theta) * cos(phi) / r
phi_y = cos(phi) * sin(theta) / r
phi_z = s_vec[2].diff(z).subs(c_to_s).simplify_full()

In [44]:
c_j

[-r*sin(phi)*sin(theta)  r*cos(phi)*cos(theta)    cos(theta)*sin(phi)]
[ r*cos(theta)*sin(phi)  r*cos(phi)*sin(theta)    sin(phi)*sin(theta)]
[                     0            -r*sin(phi)               cos(phi)]

In [28]:
h

[r^2*sin(phi)^2              0              0]
[             0            r^2              0]
[             0              0              1]

In [30]:
h_inv = h.inverse()
h_inv

[1/(r^2*sin(phi)^2)                  0                  0]
[                 0             r^(-2)                  0]
[                 0                  0                  1]

In [41]:
gamma[0][0][1]

cos(phi)/sin(phi)

In [110]:
u = SR.symbol('u', domain='real')
u_func = function('uf', nargs=3)(x, y, z)
v = SR.symbol('v', domain='real')
v_func = function('vf', nargs=3)(x, y, z)
w = SR.symbol('w', domain='real')
w_func = function('wf', nargs=3)(x, y, z)
u_vec = vector([u, v, w])
u_vec_func = vector([u_func, v_func, w_func])
u_r = SR.symbol('u_r', domain='real')
u_r_func = function('u_rf', nargs=3)(r, theta, phi)
u_theta = SR.symbol('u_theta', domain='real')
u_theta_func = function('u_thetaf', nargs=3)(r, theta, phi)
u_phi = SR.symbol('u_phi', domain='real')
u_phi_func = function('u_phif', nargs=3)(r, theta, phi)
u_r_vec = vector([u_r, u_theta, u_phi])
u_r_vec_func = vector([u_r_func, u_theta_func, u_phi_func])

In [113]:
# f(r, theta, phi) = f(s(x, y, z))
# \partial_t = f(s(x, y, z)) = f_r(r, theta, phi) s_1_t + f_theta(r, theta, phi) s_2_t + f_phi(r, theta, phi) s_3_t

-sin(phi)*sin(theta)*u_rf(r, theta, phi) - cos(phi)*sin(theta)*u_thetaf(r, theta, phi) + cos(theta)*sin(phi)*diff(u_rf(r, theta, phi), theta) + cos(phi)*cos(theta)*diff(u_thetaf(r, theta, phi), theta) - cos(theta)*u_phif(r, theta, phi) - sin(theta)*diff(u_phif(r, theta, phi), theta)

In [174]:
theta_x

-sin(theta)/(r*sin(phi))

In [102]:
type(w)

<class 'sage.symbolic.function_factory.function_factory.<locals>.NewSymbolicFunction'>

In [103]:
type(x)

<class 'sage.symbolic.expression.Expression'>

# Latitude and Longitude

In [45]:
x = SR.symbol('x', domain='real')
y = SR.symbol('y', domain='real')
z = SR.symbol('z', domain='real')
# theta - longitude
theta = SR.symbol('theta', domain='real')
# phi - latitude
phi = SR.symbol('phi', domain='real')
r = SR.symbol('r', domain='positive')
x_vec = vector([x, y, z])
r_vec = vector([theta, phi, r])
s_vec = vector([arctan(y/x), arctan(z / sqrt(x^2 + y^2)), sqrt(x^2 + y^2 + z^2)])
c_vec = vector([r * cos(phi) * cos(theta), r * cos(phi) * sin(theta), r * sin(phi)])
s_to_c = dict({r: s_vec[0], theta: s_vec[1], phi: s_vec[2]})
c_to_s = dict({x: c_vec[0], y: c_vec[1], z: c_vec[2]})
s_j = jacobian(s_vec, x_vec)
c_j = jacobian(c_vec, r_vec)
h = (c_j.transpose() * c_j).simplify_full()
kappa = sqrt(h.det())
gamma = [
    [
        [
            sum([
                1 / 2 * h_inv[alpha, k] * (h[alpha, m].diff(r_vec[n]) 
                                   + h[alpha, n].diff(r_vec[m]) 
                                   + h[m, n].diff(r_vec[alpha])) for alpha in range(3)
            ]) for k in range(3)
        ] for n in range(3)
    ] for m in range(3)
]

In [50]:
(s_j * c_j).subs(c_to_s).simplify_full()

[                     1                      0                      0]
[                     0 cos(phi)/abs(cos(phi))                      0]
[                     0                      0                      1]

In [51]:
kappa

r^2*abs(cos(phi))

In [52]:
h

[r^2*cos(phi)^2              0              0]
[             0            r^2              0]
[             0              0              1]

In [53]:
gamma[:][:][2]

[[cos(phi)^2/(r*sin(phi)^2), 0, 0], [0, 1/r, 0], [0, 0, 0]]